# Lab 12: Resting state analyses

The goal of this lab is to introduce how to do simple resting state analyses. The scripts we provide allow for doing simple seed-based analyses that generate a statistical map of functional connectivity with a seed, or to compute an ROI-ROI correlation matrix from a set of ROIs. 

For this lab some commands may be better run in the terminal rather than through the notebook. Look out for instructions on this throughout the notebook.


### Set up the computing environment

In [ ]:
%%bash

# ICA-AROMA
mkdir ${HOME}/packages
cd ${HOME}/packages
wget https://github.com/maartenmennes/ICA-AROMA/archive/v0.4.4-beta.tar.gz
tar -xf v0.4.4-beta.tar.gz
chmod +x ./ICA-AROMA-0.4.4-beta/*.py
echo 'export PATH=${HOME}/packages/ICA-AROMA-0.4.4-beta:${PATH}' >> ~/.bashrc
source ~/.bashrc

In [8]:
%%bash

# Resting state scripts
cd ${HOME}/packages
wget https://github.com/mwvoss/RestingState/archive/class_vm_scripts.zip
unzip class_vm_scripts.zip
echo 'export PATH=${HOME}/packages/RestingState-class_vm_scripts:${PATH}' >> ~/.bashrc
source ~/.bashrc

--2018-04-30 02:04:54--  https://github.com/mwvoss/RestingState/archive/class_vm_scripts.zip
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/mwvoss/RestingState/zip/class_vm_scripts [following]
--2018-04-30 02:04:54--  https://codeload.github.com/mwvoss/RestingState/zip/class_vm_scripts
Resolving codeload.github.com (codeload.github.com)... 192.30.253.120, 192.30.253.121
Connecting to codeload.github.com (codeload.github.com)|192.30.253.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20453551 (20M) [application/zip]
Saving to: ‘class_vm_scripts.zip’

     0K .......... .......... .......... .......... ..........  0%  429K 46s
    50K .......... .......... .......... .......... ..........  0%  833K 35s
   100K .......... .......... .......... .......... ..........  0% 2.57M 26s
  

In [1]:
%%bash

# create a python2.7 environment
conda create -n RestingState python=2.7
source activate RestingState


CondaValueError: prefix already exists: /home/brain/.packages/miniconda3/envs/RestingState



In [2]:
%%bash 

# Install necessary python packages

source activate RestingState
conda install -y future numpy pandas seaborn jupyter


Solving environment: ...working... done

# All requested packages already installed.



### Get octave installed (public form of matlab)

* Run the following in the terminal

In [ ]:
%%bash

# Setup debian environment for octave

echo "deb http://ftp.debian.org/debian jessie-backports main" | sudo tee -a /etc/apt/sources.list
sudo apt update
sudo apt -t jessie-backports install liboctave-dev octave

In [ ]:
%%bash

# Setup Octave

# Open Octave 
octave
# type the following in the gui
pkg install -forge image
pkg install -forge nan
# close octave (can type exit at the command prompt)

In [ ]:
%%bash

# make an rc file for octave so that it loads the package image everytime it's opened
touch ~/.octaverc
echo "pkg load image" > ~/.octaverc
echo "pkg load nan" >> ~/.octaverc

### Download example data
* Note it is in bids format, which the scripts will expect

In [3]:
%%bash

wget --quiet -O 12-Lab_data.tar.gz https://osf.io/2smk3/download &&\
tar -xf 12-Lab_data.tar.gz &&\
rm 12-Lab_data.tar.gz

tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown 

## Running the scripts 

### Make an ROI
* Let's make visual cortex ROIs from the Van Dijk review that we read
    * L MNI: -30, -88, 0
    * R MNI: 30, -88, 0 

In [12]:
%%bash

# show usage
/home/brain/packages/RestingState-class_vm_scripts/makeROI.sh


bash: makeROI.sh: No such file or directory


In [5]:
%%bash

# make an ROIs directory, the makeROI script will place ROIs there..
mkdir ROIs

# run script
/home/brain/packages/RestingState-class_vm_scripts/makeROI.sh -30 -88 0 mm 14 sphere LVis
/home/brain/packages/RestingState-class_vm_scripts/makeROI.sh 30 -88 0 mm 14 sphere RVis

-30 -88 0 mm 14 sphere LVis
The voxel coordinates are: 60 19 36
Finished!
30 -88 0 mm 14 sphere RVis
The voxel coordinates are: 30 19 36
Finished!


In [6]:
%%bash 

# take a look
fslview /usr/share/fsl/5.0/data/standard/avg152T1.nii.gz ROIs/LVis.nii.gz ROIs/RVis.nii.gz

In [7]:
%%bash

# make a list with full paths to these ROIs
find "$PWD/ROIs" -name "*.nii.gz" -type f > ${PWD}/ROIs/roilist.txt

### Run a seed-based analyses with these ROIs

Scripts assume bids storage format and provide the following workflow:
* Preprocessing: motion correction with 3dvolreg, spatial smoothing (default is 6mm FWHM)
* Denoising: ica-aroma
* Bandpass filtering: default .008 < hz < .08 
* Nuisance regression: first five eigenvariates of wm and csf, where these ROIs are determined from subject-level FAST segmentation maps which are eroded to maintain only the most probable voxels of each tissue class
* Motion scrubbing: following the nuisance regression and with volume deletion, this can allow motion to spread through volumes but avoids having to interpolate raw data
* Registration of EPI to T1: 6dof (option of BBR)
* Registration of T1 to MNI: FNIRT

In [13]:
%%bash

# you may have to edit the path for where your data are

# run this in the terminal to avoid a memory error
/home/brain/packages/RestingState-class_vm_scripts/processRestingState_bids_wrapper.sh \
--epi=/home/brain/Labs/12-Lab/sub-999/ses-activepre/func/sub-999_ses-activepre_task-rest_rec-pu_run-01_bold.nii.gz \
--roiList=${PWD}/ROIs/roilist.txt \
--compcor > output.log


Process is terminated.


### Output
* Output will go into folder named 'derivatives/rsOut' in your 12-Lab directory
    * Within this directory the bids data strcture is mirrored and directories for each session filled with output files. Output files of most interest:
    * nuisanceRegression/compcor
        * intermediate files related to the output of the nuisance regression
        * rois directory shows the nuisance ROIs
        * final denoised functional image: denoised_func_data_nonaggr_bp_res4d_normandscaled.nii 
    * seedCorrelation/compcor/rois
        * seeds mapped to functional space end with mask
        * timeseries data from residual image end with residvol_ms_ts.txt
    * EPItoT1optimized_6dof
        * registration files, these can be used to warp seeds or seedmaps between functional and MNI space or functional and T1 space
        

In [14]:
%%bash

# change directories to where your residual timeseries data are
cd /home/brain/Labs/12-Lab/derivatives/rsOut/sub-999/ses-activepre/seedCorrelation/compcor/rois/

# make a seed map with afni 3dTcorr1D tool
3dTcorr1D -prefix LVis_corrmap \
-pearson /home/brain/Labs/12-Lab/derivatives/rsOut/sub-999/ses-activepre/nuisanceRegression/compcor/denoised_func_data_nonaggr_bp_res4d_normandscaled.nii.gz \
LVis_residvol_ms_ts.txt

# convert afni brik to nifti
3dAFNItoNIFTI LVis_corrmap+orig


++ 3dTcorr1D: AFNI version=Debian-16.2.07~dfsg.1-5~nd80+1 (Jun 12 2017) [64-bit]
 + reading dataset file /home/brain/Labs/12-Lab/derivatives/rsOut/sub-999/ses-activepre/nuisanceRegression/compcor/denoised_func_data_nonaggr_bp_res4d_normandscaled.nii.gz
 + reading 1D file LVis_residvol_ms_ts.txt
 + loading dataset /home/brain/Labs/12-Lab/derivatives/rsOut/sub-999/ses-activepre/nuisanceRegression/compcor/denoised_func_data_nonaggr_bp_res4d_normandscaled.nii.gz into memory
++ Start correlations: 151552 voxels X 1 time series(300); 1 threads
++ Wrote dataset: ./LVis_corrmap+orig.BRIK
++ 3dAFNItoNIFTI: AFNI version=Debian-16.2.07~dfsg.1-5~nd80+1 (Jun 12 2017) [64-bit]


In [16]:
%%bash

# take a look, remember what to check?
cd /home/brain/Labs/12-Lab/derivatives/rsOut/sub-999/ses-activepre/seedCorrelation/compcor/rois/
fslview /home/brain/Labs/12-Lab/derivatives/rsOut/sub-999/ses-activepre/mcImgMean.nii.gz LVis_corrmap.nii

# what would you have to do to do a group analysis of functional connectivity seed maps?

# Optional practice

1) Make two new seeds from the Van Dijk paper, and generate the corresponding seedmap.

In [ ]:
# your code here

2) Use your code from lab 02 to make a timeseries plot of the resting time course from the two visual cortex ROIs, and another plot of two ROIs that should not show high resting state correlations. 

In [ ]:
# your code here